In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *

In [2]:
 sc = SparkContext()
sqlContext = SQLContext(sc)


/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [3]:
SparkContext.version

In [11]:
pwd

'/home/yogender/Desktop/NN/data_engineer_case_study/data'

In [12]:
lines = sc.textFile("./people.txt")


In [13]:
lines.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [14]:
parts = lines.map(lambda l: l.split(","))

In [15]:
parts.collect()

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]

In [20]:
from pyspark.sql import SQLContext, Row
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

In [21]:
people.collect()


[Row(name='Michael', age=29),
 Row(name='Andy', age=30),
 Row(name='Justin', age=19)]

In [22]:
schemaPeople = sqlContext.createDataFrame(people)
schemaPeople.registerTempTable("people")

/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [23]:
schemaPeople

DataFrame[name: string, age: bigint]

In [24]:
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

In [26]:
teenagers.collect()

[Row(name='Justin')]

In [28]:
parts = lines.map(lambda l: l.split(","))

In [29]:
parts.collect()

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]

In [42]:
people = parts.map(lambda p: (p[0], p[1].strip()))


In [45]:
people.collect()

[('Michael', '29'), ('Andy', '30'), ('Justin', '19')]

In [43]:
type(people)

pyspark.rdd.PipelinedRDD

In [49]:
people.collect()

[('Michael', '29'), ('Andy', '30'), ('Justin', '19')]

In [32]:
schemaString = "name age"

In [33]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]

In [47]:
fields

[StructField(name,StringType,true), StructField(age,StringType,true)]

In [35]:
schema = StructType(fields)

In [36]:
schema

StructType(List(StructField(name,StringType,true),StructField(age,StringType,true)))

In [37]:
schemaPeople = sqlContext.createDataFrame(people, schema)

In [38]:
schemaPeople.collect()

[Row(name='Michael', age='29'),
 Row(name='Andy', age='30'),
 Row(name='Justin', age='19')]

In [39]:
schemaPeople.registerTempTable("people")

In [40]:
schemaPeople.collect()

[Row(name='Michael', age='29'),
 Row(name='Andy', age='30'),
 Row(name='Justin', age='19')]

In [5]:
#sc.stop()
#sc = SparkContext()

In [6]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *

spark = sc.getOrCreate()


In [4]:
rdd_l = sc.wholeTextFiles("./*.html")

In [5]:
from pyspark.sql import SQLContext
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [6]:
rdd1 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split('\n\t')).map(lambda x: x[6]) \
.map(lambda x: x.split('prodeji')).map(lambda x: x[1]).map(lambda x: x.split(';')) \
.map(lambda x: x[0]).map(lambda x: x.strip(' '))

rows_split_addr = rdd1.map(lambda p: Row(p))

print(rows_split_addr.collect())

schemaString = "address"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
addressSchema = sqlContext.createDataFrame(rows_split_addr, schema)
#print(addressSchema.collect())
addressSchema.repartition(1).write.csv(path='./data.csv', mode="append", header="true")
addressSchema.collect()

[<Row('Masná, Praha 1 - Staré Město')>, <Row('Velvarská, Horoměřice')>, <Row('Libušina, Karlovy Vary')>, <Row('Pod Lipami, Praha 3 - Žižkov')>, <Row('Drnovská, Praha 6 - Ruzyně')>, <Row('V Průhoně, Mukařov - Žernovka')>, <Row('Mladoboleslavská, Mělník')>, <Row('Grafická, Praha 5 - Smíchov')>, <Row('Sametová, Liberec - Liberec VI-Rochlice')>, <Row('Jakubská, Praha 1 - Staré Město')>]


[Row(address='Masná, Praha 1 - Staré Město'),
 Row(address='Velvarská, Horoměřice'),
 Row(address='Libušina, Karlovy Vary'),
 Row(address='Pod Lipami, Praha 3 - Žižkov'),
 Row(address='Drnovská, Praha 6 - Ruzyně'),
 Row(address='V Průhoně, Mukařov - Žernovka'),
 Row(address='Mladoboleslavská, Mělník'),
 Row(address='Grafická, Praha 5 - Smíchov'),
 Row(address='Sametová, Liberec - Liberec VI-Rochlice'),
 Row(address='Jakubská, Praha 1 - Staré Město')]

In [8]:
addressSchema.toPandas()

,address
0,"Masná, Praha 1 - Staré Město"
1,"Velvarská, Horoměřice"
2,"Libušina, Karlovy Vary"
3,"Pod Lipami, Praha 3 - Žižkov"
4,"Drnovská, Praha 6 - Ruzyně"
5,"V Průhoně, Mukařov - Žernovka"
6,"Mladoboleslavská, Mělník"
7,"Grafická, Praha 5 - Smíchov"
8,"Sametová, Liberec - Liberec VI-Rochlice"
9,"Jakubská, Praha 1 - Staré Město"


In [6]:
header = '<meta itemprop="currency" content="CZK">'
#rdd2 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split(','))
#df = rdd2.toDF().show()
df = rdd_l.toDF(schema=["filename", "text"]).select("text")
#.flatMap(lambda x:x).filter(lambda x: x==header)
#.map(lambda x: [y.split('\n') for y in x]) \
#.filter(lambda x: (y==header for y in x))
#.map(lambda x: x.split('\n')) \
#.filter(lambda x: x==header)
#resultRDD.filter(lambda x: any(y == "OK" for _, y in x[1]))
#.map(lambda x: [x[i] for i in range(len(x))]).map(lambda x: [xi for xi in i for i not in header])
#.map(lambda x: [x[i].filter(header)] for i len(x))
#.filter(lambda X: all(val in X for val in header))
#.map(lambda x: x[13])

#.filter(lambda line: line==header)
#len(rdd2.collect())
#rdd2.collect()
#rdd2.first()
df.show()

+--------------------+
|                text|
+--------------------+
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
+--------------------+



In [7]:
from sparknlp.base import *

In [8]:
from py4j.java_gateway import java_import
#java_import(spark.jvm, "org.apache.spark.sql.api.python.*")

In [47]:
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
# make sure the next command doesn't have any error or failed downloads
# this is where the JAR and the dependencies are being downloaded
# so if you are behind the firewall, or proxy or lose internet connectivity it won't load it
# and you see that error
#spark = sparknlp.start()
#spark = SparkSession(sc)
#pipeline = PretrainedPipeline('explain_document_dl', lang='en')


spark = SparkSession.builder \
    .appName(" ")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "/home/yogender/sparknlp.jar")\
    .config("spark.driver.extraClassPath", "/home/yogender/sparknlp.jar")\
    .config("spark.executor.extraClassPath", "/home/yogender/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

'''
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.2")\
    .getOrCreate()
'''
#pipeline = PretrainedPipeline('explain_document_dl')
#documentAssembler = DocumentAssembler()

'''
documentAssembler = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document") \
.setCleanupMode("shrink")
'''



'\ndocumentAssembler = DocumentAssembler() .setInputCol("text") .setOutputCol("document") .setCleanupMode("shrink")\n'

In [300]:
rdd2.take(1)

['<!DOCTYPE html><html lang="cs" ng-app="sreality" ng-controller="MainCtrl" class="ng-scope"><head><style type="text/css">@charset "UTF-8";[ng\\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\\:form{display:block;}.ng-animate-block-transitions{transition:0s all!important;-webkit-transition:0s all!important;}.ng-hide-add-active,.ng-hide-remove{display:block!important;}</style>\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1.0,minimal-ui">\n\n\t<!--- Nastaveni meta pres JS a ne pres Angular, aby byla nastavena default hodnota pro agenty co nezvladaji PhantomJS --->\n\t<title ng:bind-template="Prodej bytu 3+kk 136&nbsp;m², Masná, Praha 1 - Staré Město • Sreality.cz" class="ng-binding">Prodej bytu 3+kk 136&nbsp;m², Masná, Praha 1 - Staré Město • Sreality.cz</title>\n\t<meta name="description" content="Byt 3+kk 136

In [301]:
from pyspark.sql.functions import udf


def soupify(rdd):
    soup = BeautifulSoup(fp, "html.parser")
    soup
    #print(soup.prettify())
    return soup

In [302]:
convertUDF = udf(lambda z: soupify(z),StringType())

In [ ]:
rdd2.withColumn("Cureated Name", upperCase(col("Name"))) \
.show(truncate=False)

In [156]:
rdd2 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split('\t\t\t\t\t'))
rdd2.collect()

[['<!DOCTYPE html><html lang="cs" ng-app="sreality" ng-controller="MainCtrl" class="ng-scope"><head><style type="text/css">@charset "UTF-8";[ng\\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\\:form{display:block;}.ng-animate-block-transitions{transition:0s all!important;-webkit-transition:0s all!important;}.ng-hide-add-active,.ng-hide-remove{display:block!important;}</style>\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1.0,minimal-ui">\n\n\t<!--- Nastaveni meta pres JS a ne pres Angular, aby byla nastavena default hodnota pro agenty co nezvladaji PhantomJS --->\n\t<title ng:bind-template="Prodej bytu 3+kk 136&nbsp;m², Masná, Praha 1 - Staré Město • Sreality.cz" class="ng-binding">Prodej bytu 3+kk 136&nbsp;m², Masná, Praha 1 - Staré Město • Sreality.cz</title>\n\t<meta name="description" content="Byt 3+kk 13

In [95]:
rdd = sc.parallelize([
    {"link0":["link1","link2"]}, {"link1":["link2","link3"]},
    {"link0":["link1","link2"]}, {"link3":["link4","link5"]}])

In [130]:
len(rdd.collect())

4

In [132]:
v = rdd.flatMap(lambda x:  [(k,tuple(v)) for k,v in x.items()] ) \


In [129]:
len(v.collect())

4

In [143]:
import re
pattern = re.compile('[\W_\']+')

In [144]:
pattern 

re.compile(r"[\W_']+", re.UNICODE)

In [9]:
rdd1 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split('\n\t')).map(lambda x: x[6]) \
.map(lambda x: x.split('prodeji')).map(lambda x: x[1]).map(lambda x: x.split(';')) \
.map(lambda x: x[0]).map(lambda x: x.strip(' '))

rows_split_addr = rdd1.map(lambda p: Row(p))

print(rows_split_addr.collect())

schemaString = "address"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
addressSchema = sqlContext.createDataFrame(rows_split_addr, schema)
#print(addressSchema.collect())
addressSchema.repartition(1).write.csv(path='./data.csv', mode="append", header="true")
addressSchema.collect()

[<Row('Masná, Praha 1 - Staré Město')>, <Row('Velvarská, Horoměřice')>, <Row('Libušina, Karlovy Vary')>, <Row('Pod Lipami, Praha 3 - Žižkov')>, <Row('Drnovská, Praha 6 - Ruzyně')>, <Row('V Průhoně, Mukařov - Žernovka')>, <Row('Mladoboleslavská, Mělník')>, <Row('Grafická, Praha 5 - Smíchov')>, <Row('Sametová, Liberec - Liberec VI-Rochlice')>, <Row('Jakubská, Praha 1 - Staré Město')>]


[Row(address='Masná, Praha 1 - Staré Město'),
 Row(address='Velvarská, Horoměřice'),
 Row(address='Libušina, Karlovy Vary'),
 Row(address='Pod Lipami, Praha 3 - Žižkov'),
 Row(address='Drnovská, Praha 6 - Ruzyně'),
 Row(address='V Průhoně, Mukařov - Žernovka'),
 Row(address='Mladoboleslavská, Mělník'),
 Row(address='Grafická, Praha 5 - Smíchov'),
 Row(address='Sametová, Liberec - Liberec VI-Rochlice'),
 Row(address='Jakubská, Praha 1 - Staré Město')]